In [ ]:
"""
This script is used to perform deconvolution with tangram

authors: Roy Oelen
"""

In [14]:
# load the libraries
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import scipy
import torch
import tangram as tg
import pickle
import scipy.sparse as sparse
import math

tg.__version__

'1.0.4'

In [3]:
# objects
class MappingResult:
    """
    Object to the results of Tangram mapping
    """
    def __init__(self, spatial_object, single_cell_object, mapping):
        """constructor
        
        Parameters
        ----------
        spatial_object : AnnData
            AnnData scanpy object of ST
        single_cell_object : AnnData
            AnnData scanpy object of SC
        mapping : AnnData
            AnnData resulting file
        
        """
        self.__spatial_object = spatial_object
        self.__single_cell_object = single_cell_object
        self.__mapping = mapping
    
    def get_spatial_object(self):
        """get ST object used for the mapping
        
        
        Returns
        -------
        result
           The spatial object used for the mapping 
        """
        return self.__spatial_object
    
    def get_single_cell_object(self):
        """get SC object used for the mapping
        
        
        Returns
        -------
        result
           The single cell object used for the mapping 
        """
        return self.__single_cell_object
    
    def get_mapping(self):
        """get the mapping result object
        
        
        Returns
        -------
        result
          The object created from doing the Tangram mapping
        """
        return self.__mapping


In [4]:
def perform_tangram_mapping(slices_dict, reference, n_genes=500, mode='cells'):
    """perform tangram mapping on a dictionary
        
    Parameters
    ----------
    slices_dict : dict
        the dictionary of ST objects to do the mapping for
    reference : AnnData
        the reference AnnData SC object to use for mapping
    n_genes : int
        the top number of most variable genes to use for the mapping
    mode : str
        the mode to use
        
    Returns
    -------
    result
       a pandas dataframe instance of the expression data for the given cell type, or None if the cell type supplied doesn't exist
    """
    # we will store the results in a dictionary
    mapped_slices = {}
    # check each slice
    for slice_name,slice_object in slices_dict.items():
        # calculate variable genes
        variable_table = sc.pp.highly_variable_genes(slice_object, inplace = False, flavor='seurat_v3', n_top_genes=n_genes)
        # select the markers
        markers = list(variable_table[(variable_table["highly_variable"] == True) & (variable_table["highly_variable_rank"] <= n_genes)].index)
        # get overlapping markers
        tg.pp_adatas(reference, slice_object, genes=markers)
        # do mapping
        ad_map = tg.map_cells_to_space(
            adata_sc=reference,
            adata_sp=slice_object,
#             device='cpu',
            device='cuda:0',
            mode=mode
        )
        tg.project_cell_annotations(ad_map, slice_object, annotation='cell_type')
        # create an object to store the result
        mapping_result = MappingResult(slice_object, reference, ad_map)
        # put in a dictionary
        mapped_slices[slice_name] = mapping_result


In [5]:
# now try with the raw data instead
slice_objects_raw = None
with open(''.join(['/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/seurat_preprocess_samples/objects/', 'spaceranger.20230823.raw.pickle']), 'rb') as f:
    slice_objects_raw = pickle.load(f)

In [6]:
# as well as the raw reference
reference_raw = None
with open('/groups/umcg-franke-scrna/tmp02/releases/blokland-2020/v1/epicardial_fat/ongoing/rtcd/references/hca/raw_expression.pickle', 'rb') as f:
    reference_raw = pickle.load(f)
# convert matrix format
reference_raw.X = reference_raw.X.tocsr()

In [ ]:
# now A1
variable_table = sc.pp.highly_variable_genes(slice_objects_raw['V10A20-016_A1'], inplace = False, flavor='seurat_v3', n_top_genes=500)
markers = list(variable_table[(variable_table["highly_variable"] == True) & (variable_table["highly_variable_rank"] <= 500)].index)
tg.pp_adatas(reference_raw, slice_objects_raw['V10A20-016_A1'], genes=markers)
ad_map = tg.map_cells_to_space(
    adata_sc=reference_raw,
    adata_sp=slice_objects_raw['V10A20-016_A1'],
#     device='cpu',
    device='cuda:0',
    mode='cells'
)
# put in object
mapping_result = MappingResult(slice_objects_raw['V10A20-016_A1'], reference_raw, ad_map)
# put in a dictionary
mapped_slices_a1 = {}
mapped_slices_a1['V10A20-016_A1'] = mapping_result
# write the result
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_A1.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(mapped_slices_a1, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# now B1
variable_table = sc.pp.highly_variable_genes(slice_objects_raw['V10A20-016_B1'], inplace = False, flavor='seurat_v3', n_top_genes=500)
markers = list(variable_table[(variable_table["highly_variable"] == True) & (variable_table["highly_variable_rank"] <= 500)].index)
tg.pp_adatas(reference_raw, slice_objects_raw['V10A20-016_B1'], genes=markers)
ad_map = tg.map_cells_to_space(
    adata_sc=reference_raw,
    adata_sp=slice_objects_raw['V10A20-016_B1'],
#     device='cpu',
    device='cuda:0',
    mode='cells'
)
# put in object
mapping_result = MappingResult(slice_objects_raw['V10A20-016_B1'], reference_raw, ad_map)
# put in a dictionary
mapped_slices_b1 = {}
mapped_slices_b1['V10A20-016_B1'] = mapping_result
# write the result
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_B1.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(mapped_slices_b1, f, pickle.HIGHEST_PROTOCOL)

In [7]:
mapping_a1 = None
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_A1.pickle', 'rb') as f:
    mapping_a1 = pickle.load(f)
# grab the result
mapping_result_a1 = mapping_a1['V10A20-016_A1']
# do the projection
tg.project_cell_annotations(mapping_result_a1.get_mapping(), mapping_result_a1.get_spatial_object(), annotation='cell_type')
# get the assignments
tangram_result_a1 = mapping_result_a1.get_spatial_object().obsm['tangram_ct_pred']
# write result
tangram_result_a1.to_csv('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/'.join(['', 'epifat_tangram_scores_a1.tsv']), sep = '\t', header = True, index = True)

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


In [8]:
mapping_b1 = None
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_B1.pickle', 'rb') as f:
    mapping_b1 = pickle.load(f)
# grab the result
mapping_result_b1 = mapping_b1['V10A20-016_B1']
# do the projection
tg.project_cell_annotations(mapping_result_b1.get_mapping(), mapping_result_b1.get_spatial_object(), annotation='cell_type')
# get the assignments
tangram_result_b1 = mapping_result_b1.get_spatial_object().obsm['tangram_ct_pred']
# write result
tangram_result_b1.to_csv('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/'.join(['', 'epifat_tangram_scores_b1.tsv']), sep = '\t', header = True, index = True)

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


In [7]:
mapping_c1 = None
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_C1.pickle', 'rb') as f:
    mapping_c1 = pickle.load(f)
# grab the result
mapping_result_c1 = mapping_c1['V10A20-016_C1']
# do the projection
tg.project_cell_annotations(mapping_result_c1.get_mapping(), mapping_result_c1.get_spatial_object(), annotation='cell_type')
# get the assignments
tangram_result_c1 = mapping_result_c1.get_spatial_object().obsm['tangram_ct_pred']
# write result
tangram_result_c1.to_csv('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/'.join(['', 'epifat_tangram_scores_c1.tsv']), sep = '\t', header = True, index = True)

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


In [17]:
# slice the object into two parts
slice_objects_raw_d1_half1 = slice_objects_raw['V10A20-016_D1'][ : math.floor(slice_objects_raw['V10A20-016_D1'].n_obs / 2), ]
slice_objects_raw_d1_half2 = slice_objects_raw['V10A20-016_D1'][math.floor(slice_objects_raw['V10A20-016_D1'].n_obs / 2) : , ]

In [21]:
# now D1, the first halve
variable_table = sc.pp.highly_variable_genes(slice_objects_raw_d1_half1, inplace = False, flavor='seurat_v3', n_top_genes=500)
markers = list(variable_table[(variable_table["highly_variable"] == True) & (variable_table["highly_variable_rank"] <= 500)].index)
tg.pp_adatas(reference_raw, slice_objects_raw_d1_half1, genes=markers)
ad_map = tg.map_cells_to_space(
    adata_sc=reference_raw,
    adata_sp=slice_objects_raw_d1_half1,
#     device='cpu',
    device='cuda:0',
    mode='cells'
)
# put in object
mapping_result = MappingResult(slice_objects_raw_d1_half1, reference_raw, ad_map)
# put in a dictionary
mapped_slices_d1_half1 = {}
mapped_slices_d1_half1['V10A20-016_D1_half1'] = mapping_result
# write the result
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_D1_half1.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(mapped_slices_d1_half1, f, pickle.HIGHEST_PROTOCOL)

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:root:500 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:12664 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 500 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.192, KL reg: 0.202
Score: 0.754, KL reg: 0.007
Score: 0.799, KL reg: 0.002
Score: 0.813, KL reg: 0.001
Score: 0.819, KL reg: 0.001
Score: 0.821, KL reg: 0.001
Score: 0.823, KL reg: 0.001
Score: 0.823, KL reg: 0.001
Score: 0.824, KL reg: 0.001
Score: 0.825, KL reg: 0.001


INFO:root:Saving results..


In [22]:
# now D1, the first second
variable_table = sc.pp.highly_variable_genes(slice_objects_raw_d1_half2, inplace = False, flavor='seurat_v3', n_top_genes=500)
markers = list(variable_table[(variable_table["highly_variable"] == True) & (variable_table["highly_variable_rank"] <= 500)].index)
tg.pp_adatas(reference_raw, slice_objects_raw_d1_half2, genes=markers)
ad_map = tg.map_cells_to_space(
    adata_sc=reference_raw,
    adata_sp=slice_objects_raw_d1_half2,
#     device='cpu',
    device='cuda:0',
    mode='cells'
)
# put in object
mapping_result = MappingResult(slice_objects_raw_d1_half2, reference_raw, ad_map)
# put in a dictionary
mapped_slices_d1_half2 = {}
mapped_slices_d1_half2['V10A20-016_D1_half2'] = mapping_result
# write the result
with open('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/raw_count_tangram_per_slices_D1_half2.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(mapped_slices_d1_half2, f, pickle.HIGHEST_PROTOCOL)

INFO:root:500 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:12669 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 500 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.196, KL reg: 0.201
Score: 0.741, KL reg: 0.006
Score: 0.787, KL reg: 0.003
Score: 0.801, KL reg: 0.001
Score: 0.807, KL reg: 0.001
Score: 0.810, KL reg: 0.001
Score: 0.811, KL reg: 0.001
Score: 0.812, KL reg: 0.001
Score: 0.812, KL reg: 0.001
Score: 0.813, KL reg: 0.001


INFO:root:Saving results..


In [23]:
# do the cell to label score for both halves
tg.project_cell_annotations(mapped_slices_d1_half2['V10A20-016_D1_half2'].get_mapping(), mapped_slices_d1_half2['V10A20-016_D1_half2'].get_spatial_object(), annotation='cell_type')
tangram_result_d1_half2 = mapped_slices_d1_half2['V10A20-016_D1_half2'].get_spatial_object().obsm['tangram_ct_pred']
tg.project_cell_annotations(mapped_slices_d1_half1['V10A20-016_D1_half1'].get_mapping(), mapped_slices_d1_half1['V10A20-016_D1_half1'].get_spatial_object(), annotation='cell_type')
tangram_result_d1_half1 = mapped_slices_d1_half1['V10A20-016_D1_half1'].get_spatial_object().obsm['tangram_ct_pred']
# concat the halves
tangram_result_d1 = pd.concat([tangram_result_d1_half1, tangram_result_d1_half2])
# save the result
tangram_result_d1.to_csv('/groups/umcg-franke-scrna/tmp02/projects/epifat/ongoing/deconvolution/tangram/results/'.join(['', 'epifat_tangram_scores_d1.tsv']), sep = '\t', header = True, index = True)

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
